한국어 Word2Vec 만들기(네이버 뉴스)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
import re
from konlpy.tag import Okt

In [15]:
data=pd.read_csv("전체새롭게합친거.csv",index_col=0)

In [16]:
data.drop(["index"],inplace=True,axis=1)
data

,기업,기사발행일,기사제목,뉴스기사본문전처리,뉴스기사 레이블
0,네이쳐글로벌,2010.10.01. 오전 9:23,"[특징주] 네이쳐글로벌 상한가…""전 대표 횡령 확인 안돼""",비메모리 반도체 업체인 네이쳐글로벌이 전 대표 횡령 혐의에 대해 확인된 바 없다고 ...,0
1,네이쳐글로벌,2010.09.28. 오후 4:29,"네이쳐글로벌, 네비스탁측이 74만여주 장내매도",네비스탁은 특별관계자로 포함된 개인투자자들이 네이쳐글로벌 주식 만주를 장내 매도했...,0
2,네이쳐글로벌,2010.09.16. 오전 9:23,"[특징주] 네이쳐글로벌, 투자경고 지정 이후 약세",네이쳐글로벌이 투자경고 지정 소식에 이어 전환사채 물량 상장에 따른 수급 불안으로 ...,0
3,네이쳐글로벌,2010.09.14. 오후 4:21,"네비스탁외 51인, 네이쳐글로벌 지분 1.26% 줄어",네비스탁외 인은 일 신규 소액주주 의결권 규합 및 보유주식수 변동에 따라 네이쳐글...,0
4,네이쳐글로벌,2010.09.16. 오후 8:08,"네이쳐글로벌, 공시번복..불성실공시법인 지정",한국거래소 코스닥시장본부는 일 네이쳐글로벌에 대해 공시번복으로 인해 불성실공시법인...,0
...,...,...,...,...,...
14807,에이치엘비,2020.09.22. 오전 5:20,"에이치엘비 리보세라닙, 글로벌 항암제로 성공 가능성↑",에이치엘비가 유럽종양학회에서 리보세라닙의 가치를 입증했다./사진에이치엘비에이치엘비의...,0
14808,에이치엘비,2020.09.21. 오후 4:18,"에이치엘비, ESMO서 '리보세라닙' 관련 논문 23개 발표",에이치엘비 김제이 기자 에이치엘비는 유럽종양학회에서 리보세라닙 관련 논문이 개...,0
14809,에이치엘비,2020.12.04. 오후 4:15,"에스퓨얼셀, 에이치엘비와 수소선박 공동개발 '맞손'…""2023년 상용화 목표""",급 수소연료전지선박 개발 협력년 수소유람선 상용화 목표설계 모듈화연료전지 전문업체 ...,0
14810,에이치엘비,2020.12.30. 오후 5:45,"코스닥, 23년만에 최고치…시총도 지각변동",코스닥지수 지난해 말 대비 . 상승개인 비중 .로 확대거래량 급등한국거래소는 일 년...,0


In [17]:
# NULL 값 존재 유무
print(data.isnull().values.any())

False


In [18]:
print(len(data))   # 기사본문 개수 출력

14812


기사의 길이 및 분포 확인

뉴스기사 레이블 시키기

In [19]:
keywords=["상폐", "적격성", "이의신청", "폐지"]
for i in range(len(data)):
    if any(keyword in str(data["뉴스기사본문전처리"][i]) for keyword in keywords)==True:
        data['뉴스기사 레이블'][i]=1
    else:
        data['뉴스기사 레이블'][i]=0


C:\Users\jangd\AppData\Local\Temp\ipykernel_27328\1452912641.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['뉴스기사 레이블'][i]=0
C:\Users\jangd\AppData\Local\Temp\ipykernel_27328\1452912641.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['뉴스기사 레이블'][i]=1


In [21]:
data[data["뉴스기사 레이블"]==0]

,기업,기사발행일,기사제목,뉴스기사본문전처리,뉴스기사 레이블
0,네이쳐글로벌,2010.10.01. 오전 9:23,"[특징주] 네이쳐글로벌 상한가…""전 대표 횡령 확인 안돼""",비메모리 반도체 업체인 네이쳐글로벌이 전 대표 횡령 혐의에 대해 확인된 바 없다고 ...,0
1,네이쳐글로벌,2010.09.28. 오후 4:29,"네이쳐글로벌, 네비스탁측이 74만여주 장내매도",네비스탁은 특별관계자로 포함된 개인투자자들이 네이쳐글로벌 주식 만주를 장내 매도했...,0
2,네이쳐글로벌,2010.09.16. 오전 9:23,"[특징주] 네이쳐글로벌, 투자경고 지정 이후 약세",네이쳐글로벌이 투자경고 지정 소식에 이어 전환사채 물량 상장에 따른 수급 불안으로 ...,0
3,네이쳐글로벌,2010.09.14. 오후 4:21,"네비스탁외 51인, 네이쳐글로벌 지분 1.26% 줄어",네비스탁외 인은 일 신규 소액주주 의결권 규합 및 보유주식수 변동에 따라 네이쳐글...,0
4,네이쳐글로벌,2010.09.16. 오후 8:08,"네이쳐글로벌, 공시번복..불성실공시법인 지정",한국거래소 코스닥시장본부는 일 네이쳐글로벌에 대해 공시번복으로 인해 불성실공시법인...,0
...,...,...,...,...,...
14807,에이치엘비,2020.09.22. 오전 5:20,"에이치엘비 리보세라닙, 글로벌 항암제로 성공 가능성↑",에이치엘비가 유럽종양학회에서 리보세라닙의 가치를 입증했다./사진에이치엘비에이치엘비의...,0
14808,에이치엘비,2020.09.21. 오후 4:18,"에이치엘비, ESMO서 '리보세라닙' 관련 논문 23개 발표",에이치엘비 김제이 기자 에이치엘비는 유럽종양학회에서 리보세라닙 관련 논문이 개...,0
14809,에이치엘비,2020.12.04. 오후 4:15,"에스퓨얼셀, 에이치엘비와 수소선박 공동개발 '맞손'…""2023년 상용화 목표""",급 수소연료전지선박 개발 협력년 수소유람선 상용화 목표설계 모듈화연료전지 전문업체 ...,0
14810,에이치엘비,2020.12.30. 오후 5:45,"코스닥, 23년만에 최고치…시총도 지각변동",코스닥지수 지난해 말 대비 . 상승개인 비중 .로 확대거래량 급등한국거래소는 일 년...,0
